## Retrieve / Transform / Visualize Stock data

In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
import yaml
import time
import pandas_datareader.data as web
import plotly.express as px

### retrieve stock data

In [3]:
with open(f"../retrieve_target.yaml", 'rb') as file:
    retrieves = yaml.safe_load(file)

In [4]:
start = datetime(2017, 1, 1)
end = datetime.today()
print(f"retrieve from {start.strftime('%Y%m%d')} to {end.strftime('%Y%m%d')}")
for k, v in retrieves["ticker"].items():
    # read ticker and source
    ticker = k
    source = v["source"]

    # retrive data
    if source == "stooq":
        stock = web.StooqDailyReader(ticker, start=start, end=end).read()
    elif source == "yahoo":
        stock = web.DataReader(ticker, "yahoo", start=start, end=end)

    # data null check
    if len(stock) == 0:
        raise ValueError("retrieved data is null")

    # save data
    stock = stock.reset_index(drop=False)
    # stock.to_csv(f"../data/{ticker}.csv", index=None)
    with open(f"../data/{ticker}.csv", "wb") as f:
        stock.to_csv(f, index=False)
    print(f"successfuly retrived: {ticker}")

    # gentle to data source
    time.sleep(1)


retrieve from 20170101 to 20220628
successfuly retrived: VOO
successfuly retrived: VWO
successfuly retrived: VEA
successfuly retrived: VTI
successfuly retrived: BND
successfuly retrived: BTC-USD
successfuly retrived: ETH-USD
successfuly retrived: DBA
successfuly retrived: USO
successfuly retrived: QQQ
successfuly retrived: VGT
successfuly retrived: VHT
successfuly retrived: VCR
successfuly retrived: XLC
successfuly retrived: VFH
successfuly retrived: XLI
successfuly retrived: VDC
successfuly retrived: VPU
successfuly retrived: XLB
successfuly retrived: XLRE
successfuly retrived: XLE


### Transform Stock data

#### transform portfolio

In [5]:
# create date master
dt_master = pd.DataFrame(columns=['Date'])
for t in retrieves["ticker"].keys():
    with open(f"../data/{ticker}.csv", "rb") as f:
        df = pd.read_csv(f)
    dt_master = pd.concat([dt_master, df.loc[:, ['Date']]], axis=0)
dt_master = dt_master.drop_duplicates(subset="Date", keep="first")
dt_master = dt_master.sort_values('Date')

In [6]:
# load portfolio settings
with open(f"../portfolio.yaml", 'rb') as file:
    portfolio = yaml.safe_load(file)

In [7]:
# create dict with each stock's number of holds
num_holds = {k: 0 for k in retrieves['ticker'].keys()}
for k in portfolio['ticker'].keys():
    num_holds[k] = 1
num_holds['BTC-USD'] = 0.01
num_holds['ETH-USD'] = 0.1

In [8]:
# create data warehouse contains every stock data
stocks = dt_master
for k in num_holds.keys():
    with open(f"../data/{k}.csv", "rb") as f:
        df = pd.read_csv(f)
    df = df.add_suffix(f'_{k}')
    df_unq = f'Date_{k}'
    stocks = pd.merge(stocks, df, how='left', left_on='Date', right_on=df_unq)

In [9]:
# select use column
cols = [f'Close_{k}' for k in num_holds.keys()]
cols.insert(0, 'Date')
stocks = stocks.loc[:, cols]

In [10]:
# calculate portfolio value
def calc_portfolio(x):
    pf = 0
    for k, v in num_holds.items():
        pf += x[f'Close_{k}'] * v
    return pf
stocks['Close_Portfolio'] = stocks.apply(lambda x: calc_portfolio(x), axis=1)

In [11]:
# convert to int
for k in num_holds.keys():
    ticker = k
    stocks[f"Close_{k}"] = np.floor(
        pd.to_numeric(stocks[f"Close_{k}"], errors="coerce")
    ).astype("Int64")
stocks["Close_Portfolio"] = np.floor(
        pd.to_numeric(stocks["Close_Portfolio"], errors="coerce")
    ).astype("Int64")

In [12]:
stocks.tail()

,Date,Close_VOO,Close_VWO,Close_VEA,Close_VTI,Close_BND,Close_BTC-USD,Close_ETH-USD,Close_DBA,Close_USO,...,Close_VCR,Close_XLC,Close_VFH,Close_XLI,Close_VDC,Close_VPU,Close_XLB,Close_XLRE,Close_XLE,Close_Portfolio
1374,2022-06-17,337,41,40,183,74,20471,1086,21,83,...,225,53,75,85,177,141,74,39,73,1266
1375,2022-06-21,346,41,40,188,74,20710,1124,21,84,...,231,54,77,86,181,143,75,39,76,1291
1376,2022-06-22,345,41,40,188,74,19987,1051,21,80,...,231,54,77,86,181,145,74,40,73,1275
1377,2022-06-23,349,41,40,189,74,21085,1143,20,78,...,234,54,76,85,183,147,73,40,70,1304
1378,2022-06-24,360,42,41,195,74,21231,1226,20,81,...,243,56,79,88,187,150,76,41,71,1343


In [13]:
# save portfolio and every stock data
stocks.to_pickle('../data/stocks.pkl')

#### transform ratio

In [15]:
# recent value ratio
recent_valid_index = stocks.dropna(subset=['Close_Portfolio']).tail(1).index.values[0]
# recent value percent
recent_values = []
recent_ratios = []
for k in portfolio["ticker"].keys():
    value = stocks.loc[recent_valid_index, f"Close_{k}"] * num_holds[k]
    ratio = value / stocks.loc[recent_valid_index, "Close_Portfolio"] * 100
    value = round(value, 2)
    ratio = round(ratio, 2)
    recent_values.append(value)
    recent_ratios.append(ratio)

In [16]:
ratio = pd.DataFrame(
    data={
        "ticker": portfolio["ticker"].keys(),
        "latest_value_sum": recent_values,
        "ratio_percent": recent_ratios,
    }
)

In [17]:
# create every stock's feature
ratio["type"] = ratio.ticker.apply(lambda x: portfolio["ticker"][x]["type"])
ratio["detail"] = ratio.ticker.apply(lambda x: portfolio["ticker"][x]["detail"])
ratio["sector"] = ratio.ticker.apply(lambda x: portfolio["ticker"][x]["sector"])
ratio["num_holds"] = ratio.ticker.apply(lambda x: num_holds[x])
ratio["latest_value"] = ratio.latest_value_sum / ratio.num_holds

In [18]:
# save every stock stats
ratio.to_pickle('../data/ratio.pkl')

In [19]:
ratio

,ticker,latest_value_sum,ratio_percent,type,detail,sector,num_holds,latest_value
0,BND,74.00,5.51,bond,VANGUARD TOTAL BOND MARKET ETF,none,1.00,74.0
1,BTC-USD,212.31,15.81,cripto,Bitcoin / USD,none,0.01,21231.0
2,ETH-USD,122.60,9.13,cripto,Etherium / USD,none,0.10,1226.0
3,VEA,41.00,3.05,stock,VANGUARD FTSE DEVELOPED MARKETS ETF,none,1.00,41.0
4,VOO,360.00,26.81,stock,VANGUARD S&P 500 ETF,none,1.00,360.0
5,VTI,195.00,14.52,stock,VANGUARD TOTAL STOCK MARKET ETF,none,1.00,195.0
6,VWO,42.00,3.13,stock,VANGUARD FTSE EMERGING MARKETS ETF,none,1.00,42.0
7,QQQ,294.00,21.89,stock,Invesco QQQ ETF,IT,1.00,294.0


#### transform sharpe ratio

In [20]:
# calc sharpe ratio
sharpe = stocks.loc[:, ["Date", "Close_Portfolio"]]
sharpe = sharpe.dropna(subset=["Close_Portfolio"])
sharpe["lag_1d"] = sharpe["Close_Portfolio"].shift(1)
sharpe["rate_change"] = np.log(sharpe.Close_Portfolio / sharpe.lag_1d)
sharpe["one_year_mean"] = sharpe["rate_change"].rolling(252).mean()
sharpe["one_year_std"] = sharpe["rate_change"].rolling(252).std()
sharpe["sharpe_ratio"] = sharpe.one_year_mean / sharpe.one_year_std
sharpe["sharpe_ratio_annual"] = sharpe["sharpe_ratio"] * 252**0.5
sharpe = sharpe.dropna(subset=["sharpe_ratio_annual"])
sharpe = sharpe.loc[:, ["Date", "sharpe_ratio_annual"]]

In [21]:
sharpe.tail()

,Date,sharpe_ratio_annual
1374,2022-06-17,-0.834188
1375,2022-06-21,-0.671261
1376,2022-06-22,-0.730804
1377,2022-06-23,-0.705313
1378,2022-06-24,-0.646594


In [23]:
# save sharpe ratio
sharpe.to_pickle('../data/sharpe.pkl')

### Visualize

In [22]:
# plot sharpe ratio
fig = px.line(sharpe.dropna(subset=['sharpe_ratio_annual']), x="Date", y="sharpe_ratio_annual")
fig.add_hline(1, line_color="red")
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()

In [24]:
# plot portfolio
fig = px.line(stocks.dropna(subset=['Close_Portfolio']), x="Date", y="Close_Portfolio")
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()

In [25]:
# plot ratio
fig = px.sunburst(
ratio,
path=["type", "ticker"],
values="ratio_percent",
title="Portfolio Recent Value Ratio",
)
fig.show()